In [78]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from sklearn.model_selection import train_test_split
import random
import re

pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_colwidth', 20)
pd.set_option('display.width', 20)

data = pd.read_csv("C:\\Users\\marci\\Downloads\\real_estate_ads 3.csv")

def process_params(params_str):
    params = {}
    values = params_str.split('<br>')
    for value in values:
        if '<=>' in value:
            param, val = value.split('<=>')
            # Sprawdź czy parametr ma więcej niż jedną wartość
            if '<->' in val:
                val = val.split('<->')
            params[param] = val
    return params
    
# Tworzenie ramki danych z przetworzonymi parametrami
df_params = data['params'].apply(process_params).apply(pd.Series)

# Dodanie kolumn z ramki danych df_params do ramki danych data
data_new = pd.concat([data, df_params], axis=1)

# Usunięcie zduplikowanych kolumn
data_new = data_new.loc[:, ~data_new.columns.duplicated()]

In [79]:
target_columns = ['rent[currency]', 'building_ownership', 'media_types', 'security_types',
                  'equipment_types', 'extras_types', 'remote_services', 'market', 'rooms_num', 'building_type',
                  'floor_no', 'building_floors_num', 'building_material', 'windows_type', 'heating', 'rent',
                  'no_rooms', 'construction_status']

unique_values = {}


for col in target_columns:

    if data_new[col].apply(type).eq(list).any():
        data_new[col] = data_new[col].apply(lambda x: ','.join(x))

    unique_values[col] = data_new[col].unique()

data_new = data_new.drop('params', axis=1)

prob_columns = ['extras_types', 'equipment_types', 'media_types', 'security_types']

def process_values(value):
    # Sprawdź czy wartość zawiera przecinki
    if ',' in value:
        # Podziel wartość na fragmenty
        parts = value.split(',')
        # Usuń spacje wokół każdej części
        parts = [part.strip() for part in parts]
        # Sprawdź czy każda część ma tylko jedną wartość
        if all(len(part) == 1 for part in parts):
            # Jeśli tak, złącz części w jedną wartość
            processed_value = ''.join(parts)
        else:
            # W przeciwnym razie zachowaj oryginalną wartość
            processed_value = value
    else:
        # Jeśli nie ma przecinków, zachowaj oryginalną wartość
        processed_value = value
    return processed_value



for col in prob_columns:
    data_new[col] = data_new[col].apply(process_values)

In [80]:
def process_values(value):
    if pd.notna(value) and isinstance(value, str) and ',' in value:
        parts = value.split(',')
        parts = [part.strip() for part in parts]
        parts.sort()  # Sortowanie alfabetyczne
        processed_value = ','.join(parts)
    else:
        processed_value = value
    return processed_value

# Przetwarzanie wartości dla kolumn w prob_columns
for col in prob_columns:
    data_new[col] = data_new[col].apply(process_values)

In [81]:
data_new['rent'] = pd.to_numeric(data_new['rent'], errors='coerce')
def convert_currency(row):
    if row['rent[currency]'] == 'USD':
        return row['rent'] * 3.94
    elif row['rent[currency]'] == 'EUR':
        return row['rent'] * 4.29
    else:
        return row['rent']

# Zastosowanie funkcji do ramki danych
data_new['rent'] = data_new.apply(convert_currency, axis=1)

data_new = data_new.drop(['rent[currency]', 'no_rooms'],axis=1)

In [82]:
data_new['market'] = data_new['market'].replace({'primary': 1, 'secondary': 2})
data_new['rooms_num'] = data_new['rooms_num'].replace({'more': 8})
data_new['building_type'] = data_new['building_type'].replace({'block': 1, 'apartment': 2, 'ribbon': 3, 'tenement': 4, 'house': 5})
data_new['floor_no'] = data_new['floor_no'].replace({'ground_floor': 0, 
                                                 'floor_1': 1, 
                                                 'floor_2': 2, 
                                                 'floor_3': 3, 
                                                 'floor_4': 4,
                                                 'floor_5': 5,
                                                 'floor_6': 6,
                                                 'floor_7': 7,
                                                 'floor_8': 8,
                                                 'floor_9': 9,
                                                 'floor_10': 10,
                                                 'floor_higher_10':11,
                                                 'cellar': -1,
                                                 'garret': 20})
data_new['building_floors_num'] = pd.to_numeric(data_new['building_floors_num'], errors='coerce')
data_new['building_floors_num'] = data_new['building_floors_num'].replace(110, 10)
data_new['building_material'] = data_new['building_material'].replace({'brick': 1, 'concrete_plate': 2, 'other': 3, 'breezeblock': 4, 'reinforced_concrete': 5,
                                                                  'silikat':6, 'concrete': 7, 'cellular_concrete': 8, 'hydroton': 9, 'wood': 10})
data_new['windows_type'] = data_new['windows_type'].replace({'plastic': 1, 'wooden': 2, 'aluminium': 3})
data_new['heating'] = data_new['heating'].replace({'urban': 1, 'other': 2, 'gas': 3, 'boiler_room': 4, 'electrical': 5, 'tiled_stove': 6})
data_new['construction_status'] = data_new['construction_status'].replace({'to_completion': 1, 'ready_to_use': 2, 'to_renovation': 3})


In [83]:
data_new.columns

Index(['id',
       'market',
       'created_at_first',
       'created_at',
       'district_lon',
       'district_lat',
       'title',
       'description',
       'price',
       'm',
       'price_per_m',
       'map_lon',
       'map_lat',
       'price[currency]',
       'rooms_num',
       'building_type',
       'floor_no',
       'building_floors_num',
       'building_material',
       'windows_type',
       'heating',
       'build_year',
       'construction_status',
       'rent',
       'building_ownership',
       'free_from',
       'media_types',
       'security_types',
       'equipment_types',
       'extras_types',
       'remote_services'],
      dtype='object')

In [84]:
data_new[['media_types',
       'security_types',
       'equipment_types',
       'extras_types']].head()

,media_types,security_types,equipment_types,extras_types
0,cable-television...,,,"balcony,separate..."
1,cable-television...,,,balcony
2,,entryphone,,"balcony,basement"
3,,entryphone,,"balcony,basement"
4,cable-television...,entryphone,"fridge,furniture...",basement


In [85]:
data_new['extras_types'].value_counts()

extras_types
balcony,lift                                 1951
balcony,garage,lift                          1594
                                             1111
balcony,basement,separate_kitchen             629
balcony,basement,lift                         613
                                             ... 
balcony,two_storey                              1
basement,garden,lift,terrace                    1
air_conditioning,balcony,separate_kitchen       1
garden,lift,terrace,usable_room                 1
air_conditioning                                1
Name: count, Length: 265, dtype: int64

In [86]:
# Uzyskaj listę unikatowych wartości w kolumnie 'extras_types'
unique_values = data_new['extras_types'].str.split(',').explode().unique()

# Utwórz listę wszystkich unikalnych elementów
all_unique_elements = list(unique_values)

# Utwórz słownik
extras_types_dict = {element: [] for element in all_unique_elements}

# Wyświetl uzyskany słownik
print(extras_types_dict)

{'balcony': [], 'separate_kitchen': [], 'basement': [], 'lift': [], '': [], 'garage': [], 'garden': [], 'air_conditioning': [], 'terrace': [], 'usable_room': [], '0': [], 'two_storey': []}


In [87]:
df_extras = pd.DataFrame()

# Iteruj przez każdą wartość w kolumnie 'extras_types'
for col in extras_types_dict:
    # Jeśli kolumna 'extras_types' zawiera daną wartość, ustaw 1, w przeciwnym razie 0
    df_extras[col] = data_new['extras_types'].str.contains(col, na=False).astype(int)

# Wyświetl utworzoną ramkę danych
print(df_extras.columns)
df_extras.head()

Index(['balcony',
       'separate_kitchen',
       'basement',
       'lift', '',
       'garage',
       'garden',
       'air_conditioning',
       'terrace',
       'usable_room',
       '0',
       'two_storey'],
      dtype='object')


,balcony,separate_kitchen,basement,lift,,garage,garden,air_conditioning,terrace,usable_room,0,two_storey
0,1,1,0,0,1,0,0,0,0,0,0,0
1,1,0,0,0,1,0,0,0,0,0,0,0
2,1,0,1,0,1,0,0,0,0,0,0,0
3,1,0,1,0,1,0,0,0,0,0,0,0
4,0,0,1,0,1,0,0,0,0,0,0,0


In [88]:
df_extras.drop(df_extras.columns[4],axis=1, inplace=True)
df_extras.drop(df_extras.columns[9],axis=1,inplace=True)
df_extras.columns

Index(['balcony',
       'separate_kitchen',
       'basement',
       'lift',
       'garage',
       'garden',
       'air_conditioning',
       'terrace',
       'usable_room',
       'two_storey'],
      dtype='object')

In [89]:
data_new.drop('extras_types', axis=1, inplace=True)

# Połącz ramkę danych data_new z df_extras
data_new = pd.concat([data_new, df_extras], axis=1)

In [90]:
data_new.columns

Index(['id',
       'market',
       'created_at_first',
       'created_at',
       'district_lon',
       'district_lat',
       'title',
       'description',
       'price',
       'm',
       'price_per_m',
       'map_lon',
       'map_lat',
       'price[currency]',
       'rooms_num',
       'building_type',
       'floor_no',
       'building_floors_num',
       'building_material',
       'windows_type',
       'heating',
       'build_year',
       'construction_status',
       'rent',
       'building_ownership',
       'free_from',
       'media_types',
       'security_types',
       'equipment_types',
       'remote_services',
       'balcony',
       'separate_kitchen',
       'basement',
       'lift',
       'garage',
       'garden',
       'air_conditioning',
       'terrace',
       'usable_room',
       'two_storey'],
      dtype='object')

In [91]:
unique_values = data_new['media_types'].str.split(',').explode().unique()

# Utwórz listę wszystkich unikalnych elementów
all_unique_elements = list(unique_values)

# Utwórz słownik
media_types_dict = {element: [] for element in all_unique_elements}

# Wyświetl uzyskany słownik
print(media_types_dict)

df_media = pd.DataFrame()

# Iteruj przez każdą wartość w kolumnie 'media_types'
for col in media_types_dict:
    # Jeśli kolumna 'extras_types' zawiera daną wartość, ustaw 1, w przeciwnym razie 0
    df_media[col] = data_new['media_types'].str.contains(col, na=False).astype(int)

df_media.drop(df_media.columns[3],axis=1, inplace=True)
df_media.drop(df_media.columns[3],axis=1, inplace=True)

data_new.drop('media_types', axis=1, inplace=True)

# Połącz ramkę danych data_new z df_media
data_new = pd.concat([data_new, df_media], axis=1)
print(df_media.columns)
data_new.columns

{'cable-television': [], 'internet': [], 'phone': [], '': [], '0': []}
Index(['cable-television',
       'internet',
       'phone'],
      dtype='object')


Index(['id',
       'market',
       'created_at_first',
       'created_at',
       'district_lon',
       'district_lat',
       'title',
       'description',
       'price',
       'm',
       'price_per_m',
       'map_lon',
       'map_lat',
       'price[currency]',
       'rooms_num',
       'building_type',
       'floor_no',
       'building_floors_num',
       'building_material',
       'windows_type',
       'heating',
       'build_year',
       'construction_status',
       'rent',
       'building_ownership',
       'free_from',
       'security_types',
       'equipment_types',
       'remote_services',
       'balcony',
       'separate_kitchen',
       'basement',
       'lift',
       'garage',
       'garden',
       'air_conditioning',
       'terrace',
       'usable_room',
       'two_storey',
       'cable-television',
       'internet',
       'phone'],
      dtype='object')

In [92]:
unique_values = data_new['equipment_types'].str.split(',').explode().unique()

# Utwórz listę wszystkich unikalnych elementów
all_unique_elements = list(unique_values)

# Utwórz słownik
eq_types_dict = {element: [] for element in all_unique_elements}

# Wyświetl uzyskany słownik
print(eq_types_dict)

df_eq = pd.DataFrame()

# Iteruj przez każdą wartość w kolumnie 'media_types'
for col in eq_types_dict:
    # Jeśli kolumna 'extras_types' zawiera daną wartość, ustaw 1, w przeciwnym razie 0
    df_eq[col] = data_new['equipment_types'].str.contains(col, na=False).astype(int)

df_eq.drop(df_eq.columns[8],axis=1, inplace=True)
df_eq.drop(df_eq.columns[0],axis=1, inplace=True)

data_new.drop('equipment_types', axis=1, inplace=True)

# Połącz ramkę danych data_new z df_media
data_new = pd.concat([data_new, df_eq], axis=1)
print(df_eq.columns)
data_new.columns

{'': [], 'fridge': [], 'furniture': [], 'stove': [], 'washing_machine': [], 'oven': [], 'dishwasher': [], 'tv': [], '0': []}
Index(['fridge',
       'furniture',
       'stove',
       'washing_machine',
       'oven',
       'dishwasher',
       'tv'],
      dtype='object')


Index(['id',
       'market',
       'created_at_first',
       'created_at',
       'district_lon',
       'district_lat',
       'title',
       'description',
       'price',
       'm',
       'price_per_m',
       'map_lon',
       'map_lat',
       'price[currency]',
       'rooms_num',
       'building_type',
       'floor_no',
       'building_floors_num',
       'building_material',
       'windows_type',
       'heating',
       'build_year',
       'construction_status',
       'rent',
       'building_ownership',
       'free_from',
       'security_types',
       'remote_services',
       'balcony',
       'separate_kitchen',
       'basement',
       'lift',
       'garage',
       'garden',
       'air_conditioning',
       'terrace',
       'usable_room',
       'two_storey',
       'cable-television',
       'internet',
       'phone',
       'fridge',
       'furniture',
       'stove',
       'washing_machine',
       'oven',
       'dishwasher',
       'tv'],
      

In [93]:
unique_values = data_new['security_types'].str.split(',').explode().unique()

# Utwórz listę wszystkich unikalnych elementów
all_unique_elements = list(unique_values)

# Utwórz słownik
sec_types_dict = {element: [] for element in all_unique_elements}

# Wyświetl uzyskany słownik
print(sec_types_dict)

df_sec = pd.DataFrame()

# Iteruj przez każdą wartość w kolumnie 'media_types'
for col in sec_types_dict:
    # Jeśli kolumna 'extras_types' zawiera daną wartość, ustaw 1, w przeciwnym razie 0
    df_sec[col] = data_new['security_types'].str.contains(col, na=False).astype(int)

df_sec.drop(df_sec.columns[2],axis=1, inplace=True)
df_sec.drop(df_sec.columns[0],axis=1, inplace=True)

data_new.drop('security_types', axis=1, inplace=True)

# Połącz ramkę danych data_new z df_media
data_new = pd.concat([data_new, df_sec], axis=1)
print(df_sec.columns)
data_new.columns

{'': [], 'entryphone': [], '0': [], 'closed_area': [], 'monitoring': [], 'anti_burglary_door': [], 'roller_shutters': [], 'alarm': []}
Index(['entryphone',
       'closed_area',
       'monitoring',
       'anti_burglary_door',
       'roller_shutters',
       'alarm'],
      dtype='object')


Index(['id',
       'market',
       'created_at_first',
       'created_at',
       'district_lon',
       'district_lat',
       'title',
       'description',
       'price',
       'm',
       'price_per_m',
       'map_lon',
       'map_lat',
       'price[currency]',
       'rooms_num',
       'building_type',
       'floor_no',
       'building_floors_num',
       'building_material',
       'windows_type',
       'heating',
       'build_year',
       'construction_status',
       'rent',
       'building_ownership',
       'free_from',
       'remote_services',
       'balcony',
       'separate_kitchen',
       'basement',
       'lift',
       'garage',
       'garden',
       'air_conditioning',
       'terrace',
       'usable_room',
       'two_storey',
       'cable-television',
       'internet',
       'phone',
       'fridge',
       'furniture',
       'stove',
       'washing_machine',
       'oven',
       'dishwasher',
       'tv',
       'entryphone',
       'clo

In [94]:
data_new.head()

,id,market,created_at_first,created_at,district_lon,district_lat,title,description,price,m,...,washing_machine,oven,dishwasher,tv,entryphone,closed_area,monitoring,anti_burglary_door,roller_shutters,alarm
0,49341,2,2018-01-04 13:11:05,2018-01-08 11:27:35,16.9408,52.38356,NaN,<p>Zapraszam do ...,289000.0,48.000000,...,0,0,0,0,0,0,0,0,0,0
1,261720,2,2018-01-15 19:12:07,2018-01-25 11:11:31,16.9408,52.38356,NaN,<p>Zapraszam do ...,379000.0,61.000000,...,0,0,0,0,0,0,0,0,0,0
2,50441,2,2018-01-04 13:53:29,2018-01-11 05:55:12,16.9408,52.38356,Rataje Nowe Mias...,<p>Weychan Nieru...,339000.0,63.000000,...,0,0,0,0,1,0,0,0,0,0
3,203518,2,2018-01-12 11:10:57,2018-01-30 14:38:39,16.9408,52.38356,Mieszkanie 3 pok...,<p>Oferuję na sp...,330000.0,59.200001,...,0,0,0,0,1,0,0,0,0,0
4,5,2,2018-01-02 11:34:51,2018-01-18 14:22:29,16.9408,52.38356,Inwestycja -rond...,<p><strong>Dwa p...,259000.0,38.000000,...,1,0,0,0,1,0,0,0,0,0


In [99]:
data_new.isna().sum().sort_values(ascending=False)

rent                   9898
remote_services        9520
building_floors_num    2088
title                   927
district_lon            187
                       ... 
construction_status       0
building_ownership        0
free_from                 0
market                    0
alarm                     0
Length: 53, dtype: int64

In [100]:
data_new.to_csv('pojebana_csvka_go_craaaaazy.csv', index=False)